As we saw last week, Profits are rolling in nicely from Chin & Beard Suds Co. but all is not perfect at the company. Recently our Suds shops have been the victim of a number of thefts. Thankfully our systems allow our stores to record the thefts, when the theft occurred and when we adjusted the inventory to reflect the reduced amounts.

The data has all the parts we need to answer a few questions:
- What product is stolen the most?
- How many items of stock haven't been updated in the inventory levels yet? 
- What stores need to update their inventory levels?
- Which store is the fastest at updating inventory levels post a theft?
- Which stores have updated their stock levels incorrectly?
- To be able to answer these questions, we need you to create the following data set.

# Requirements:

<img src="https://2.bp.blogspot.com/-3wWkmbh6PAE/XKPYobEzaiI/AAAAAAAAAL0/fWDLhPpfROM1fb8WOwBlDu9BNn3z7Ul4QCLcBGAs/s320/Input.JPG" alt="Alternative text" />

Input data from both sheets
- Update Store IDs to use the Store Names
- Clean up the Product Type to just return two products types: Bar and Liquid
- Measure the difference in days between when the theft occurred and when the stock was updated
- Measure the variance in stolen stock and inventory adjustment 
- *You will likely need Prep Builder v2019.1 as I used Rows to Columns pivot but you lot are innovative so who knows?*

# Output:

<img src="https://1.bp.blogspot.com/-J1NMZM4f0aw/XKPX5Pgq8AI/AAAAAAAAALs/7aTu-2b84OQf2sbgzyu-lqIEaaYSRG_tQCLcBGAs/s400/Output.JPG" alt="Alternative text" />

- 8 Rows of data (9 including the header)
- 8 Columns of data
- 4 Null cells due to inventory updates not occurring
- One Row per Crime Refernce Number

In [383]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [384]:
df1 = pd.read_excel('Week 8 Input.xlsx', sheet_name='Theft Audit')
df1.head()

,Type,Action,Date,Quantity,Store ID,Crime Ref Number
0,Bar,Theft,2019-03-19,10.0,OX1,S04P1
1,Bar,Stock Adjusted,2019-03-25,-10.0,OX1,S04P1
2,Bar,Theft,2019-03-22,5.0,OX1,S04P2
3,Bar,Stock Adjusted,2019-03-25,-4.0,OX1,S04P2
4,Liquid,Theft,2019-03-02,100.0,WM1,S04P3


In [385]:
df2 = pd.read_excel('Week 8 Input.xlsx', sheet_name='Branch ID')
df2.head()

,Branch ID
0,OX1 - Oxford Street
1,WM1 - Wimbledon 1
2,WM2 - Wimbledon 2
3,ST1 - Stratford


In [386]:
df2[['Store ID', 'Branch Name']] = df2['Branch ID'].str.split(' - ', expand=True)
df2.drop(columns=['Branch ID'], inplace=True)
df2.head()

,Store ID,Branch Name
0,OX1,Oxford Street
1,WM1,Wimbledon 1
2,WM2,Wimbledon 2
3,ST1,Stratford


In [387]:
print(df1['Type'].unique())

['Bar' 'Liquid' 'Luquid' 'Soap Bar']


In [388]:
for index, series in df1.iterrows():
    if 'Bar' in df1.loc[index,'Type']:
        df1.loc[index,'Type'] = 'Bar'
    else:
        df1.loc[index,'Type'] = 'Liquid'
        
df1.head(100)

,Type,Action,Date,Quantity,Store ID,Crime Ref Number
0,Bar,Theft,2019-03-19,10.0,OX1,S04P1
1,Bar,Stock Adjusted,2019-03-25,-10.0,OX1,S04P1
2,Bar,Theft,2019-03-22,5.0,OX1,S04P2
3,Bar,Stock Adjusted,2019-03-25,-4.0,OX1,S04P2
4,Liquid,Theft,2019-03-02,100.0,WM1,S04P3
5,Liquid,Stock Adjusted,2019-03-02,-100.0,WM1,S04P3
6,Bar,Theft,2019-04-03,5.0,WM2,S04P4
7,Liquid,Theft,2019-03-22,14.0,OX1,S04P5
8,Liquid,Theft,2019-02-02,2.0,WM2,S04P6
9,Liquid,Stock Adjusted,2019-04-03,-2.0,WM2,S04P6


In [389]:
df = pd.merge(df1, df2, how='left', on='Store ID')
df.head()

,Type,Action,Date,Quantity,Store ID,Crime Ref Number,Branch Name
0,Bar,Theft,2019-03-19,10.0,OX1,S04P1,Oxford Street
1,Bar,Stock Adjusted,2019-03-25,-10.0,OX1,S04P1,Oxford Street
2,Bar,Theft,2019-03-22,5.0,OX1,S04P2,Oxford Street
3,Bar,Stock Adjusted,2019-03-25,-4.0,OX1,S04P2,Oxford Street
4,Liquid,Theft,2019-03-02,100.0,WM1,S04P3,Wimbledon 1


In [390]:
df = df.pivot_table(values='Quantity', index=['Type','Date','Store ID','Crime Ref Number','Branch Name'], columns='Action')
df.reset_index(inplace=True)
# df.set_index(['Type','Date','Store ID','Crime Ref Number','Branch Name'], drop=True).unstack('Action')
# df.reindex(['Stock Adjusted','Theft'], axis=1)
df.head()

Action,Type,Date,Store ID,Crime Ref Number,Branch Name,Stock Adjusted,Theft
0,Bar,2019-02-28,WM1,S04P7,Wimbledon 1,NaN,22.0
1,Bar,2019-03-01,WM1,S04P7,Wimbledon 1,-20.0,NaN
2,Bar,2019-03-19,OX1,S04P1,Oxford Street,NaN,10.0
3,Bar,2019-03-22,OX1,S04P2,Oxford Street,NaN,5.0
4,Bar,2019-03-25,OX1,S04P1,Oxford Street,-10.0,NaN


In [391]:
theft_df = df[df['Theft'].notnull()]
theft_df.reset_index(inplace=True, drop=True)
theft_df.drop(columns=['Stock Adjusted'], inplace=True)
theft_df.rename(columns={'Theft':'Stolen Volume', 'Date':'Theft'}, inplace=True)
theft_df.head()

Action,Type,Theft,Store ID,Crime Ref Number,Branch Name,Stolen Volume
0,Bar,2019-02-28,WM1,S04P7,Wimbledon 1,22.0
1,Bar,2019-03-19,OX1,S04P1,Oxford Street,10.0
2,Bar,2019-03-22,OX1,S04P2,Oxford Street,5.0
3,Bar,2019-04-03,WM2,S04P4,Wimbledon 2,5.0
4,Liquid,2019-02-02,WM2,S04P6,Wimbledon 2,2.0


In [392]:
stock_adjust_df = df[df['Theft'].isnull()]
stock_adjust_df.reset_index(inplace=True, drop=True)
stock_adjust_df.drop(columns=['Theft', 'Branch Name', 'Store ID', 'Type'], inplace=True)
stock_adjust_df.rename(columns={'Stock Adjusted':'Adjusted Volume', 'Date': 'Adjust Date'}, inplace=True)
stock_adjust_df.head()

Action,Adjust Date,Crime Ref Number,Adjusted Volume
0,2019-03-01,S04P7,-20.0
1,2019-03-25,S04P1,-10.0
2,2019-03-25,S04P2,-4.0
3,2019-03-01,S04P8,-105.0
4,2019-04-03,S04P6,-2.0


In [393]:
result = pd.merge(theft_df, stock_adjust_df, how='outer', on='Crime Ref Number')
result['Adjusted Volume'] = result['Adjusted Volume'].fillna(0)
result['Stock Variance'] = result['Stolen Volume'] + result['Adjusted Volume']
result.head(20)

Action,Type,Theft,Store ID,Crime Ref Number,Branch Name,Stolen Volume,Adjust Date,Adjusted Volume,Stock Variance
0,Bar,2019-02-28,WM1,S04P7,Wimbledon 1,22.0,2019-03-01,-20.0,2.0
1,Bar,2019-03-19,OX1,S04P1,Oxford Street,10.0,2019-03-25,-10.0,0.0
2,Bar,2019-03-22,OX1,S04P2,Oxford Street,5.0,2019-03-25,-4.0,1.0
3,Bar,2019-04-03,WM2,S04P4,Wimbledon 2,5.0,NaT,0.0,5.0
4,Liquid,2019-02-02,WM2,S04P6,Wimbledon 2,2.0,2019-04-03,-2.0,0.0
5,Liquid,2019-02-27,OX1,S04P8,Oxford Street,105.0,2019-03-01,-105.0,0.0
6,Liquid,2019-03-02,WM1,S04P3,Wimbledon 1,100.0,NaT,0.0,100.0
7,Liquid,2019-03-22,OX1,S04P5,Oxford Street,14.0,NaT,0.0,14.0
